In [ ]:
# ============================
# STEP 1: MOUNT GOOGLE DRIVE
# ============================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# Change directory to your project
PROJECT_PATH = '/content/drive/Othercomputers/Laptop/0a.Nathan Model'
%cd "$PROJECT_PATH"

# ============================
# STEP 2: INSTALL DEPENDENCIES
# ============================
# Install requirements if you have a requirements.txt file
!pip install -qr "colab_requirements.txt"
#!pip install -q pyarrow==14.0.2 transformers[torch] datasets evaluate

# Check if torch is using GPU
import torch
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# ============================
# STEP 3: RUN YOUR NLP CODE
# ============================
print("Running model...")
!python start.py
#testing basic og function, tf rels = False => 87,71,74 full
#then add neg sampling before binary filter, tf rels = False => 87,70,72 3000   (74.35, 57.78, 65.03)
#span neg sampling and add rel neg sampling, tf rels = False => 88, 69, 71 => rel pr is low => I think if you neg sample the rels, you must teacher force the rels also
#span neg sampling, rel neg sampling, add prune rels after rel neg sampling to save space, tf rels = False  => this crashes, need to fix the rel rep code
#span neg sampling, rel neg sampling, tf rels = True   88,71,73   3000

#from what I see neg sampling ont eh spans only works with no tf on teh rels and no neg sampling on the rels

# BE tagger => span neg sampling => span filter with tagger scores => rel gen   86,68,70 3000  => uses less memory
# span neg sampling => BE tagger => span filter with tagger scores => rel gen   86,66,67 3000
# BE tagger => span neg sampling => span filter with tths scores => rel-neg-sampling => rel gen  87,70,72 3000
# BECO tagger => span neg sampling => span filter with tagger scores => rel-neg-sampling => rel gen 87,69,69 3000
# BE tagger => span neg sampling => span filter with tths scores => rel-neg-sampling => tf rels => rel gen  86,70,70 3000 => takes longer to train but will get there...
# BE tagger => span filter with tths scores => rel gen

# BE tagger => span filter with tths scores => span marker => rel gen   => 85,70,70 => so marking doesn't really help at all, go figure....

Mounted at /content/drive
/content/drive/Othercomputers/Laptop/0a.Nathan Model
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━